[View in Colaboratory](https://colab.research.google.com/github/JohnnyUrosevic/Machine-Learning-Shenanigans/blob/master/keras_kaggle_digits.ipynb)

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

!pip3 install hyperas
!pip3 install networkx==1.11

from google.colab import files

Download Files from google drive, and add this file to the temporary local system so it can be recognized by Hyperas

In [2]:
from google.colab import auth
from googleapiclient.discovery import build
import io , requests, os
import sys
auth.authenticate_user()
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

SOURCE_FOLDER='/content/datalab/'


def get_parent_folder(folder_name):
  page_token = None
  folder_array = []
  query = "name='%s' and mimeType='application/vnd.google-apps.folder'" % folder_name
  while True:
      response = drive_service.files().list(q=query,
                                          spaces='drive',
                                          fields='nextPageToken, files(id, name)',
                                          pageToken=page_token).execute()
      for file in response.get('files', []):
          # Process change
          #print (file.get('name'), file.get('id'))
          folder_array.append({"name" : file.get('name'), "id" : file.get('id')})
      page_token = response.get('nextPageToken', None)
      if page_token is None:
          break
  return folder_array


def get_files_from_parent(parent_id):
  page_token = None
  folder_array = dict()
  query = "'%s' in parents" % parent_id
  while True:
      response = drive_service.files().list(q=query,
                                          spaces='drive',
                                          fields='nextPageToken, files(id, name)',
                                          pageToken=page_token).execute()
      for file in response.get('files', []):
          # Process change
          #print (file.get('name'), file.get('id'))
          folder_array.update({file.get('name'):file.get('id')})
      page_token = response.get('nextPageToken', None)
      if page_token is None:
          break
  return folder_array

def get_file_buffer(file_id, verbose=0):
  from googleapiclient.http import MediaIoBaseDownload
  request = drive_service.files().get_media(fileId=file_id)
  downloaded = io.BytesIO()
  downloader = MediaIoBaseDownload(downloaded, request)
  done = False
  while done is False:
    # _ is a placeholder for a progress object that we ignore.
    # (Our file is small, so we skip reporting progress.)
    progress, done = downloader.next_chunk()
    if verbose:
      sys.stdout.flush()
      sys.stdout.write('\r')
      percentage_done = progress.resumable_progress * 100/progress.total_size
      sys.stdout.write("[%-100s] %d%%" % ('='*int(percentage_done), int(percentage_done)))
  downloaded.seek(0)
  return downloaded

parent_folder = get_parent_folder('Kaggle Digit')

input_file_meta = get_files_from_parent(parent_folder[0]["id"])


for file, id in input_file_meta.items():
  downloaded = get_file_buffer(id, verbose=1)
  dest_file = os.path.join(SOURCE_FOLDER, file)
  print("processing %s data" % file)
  with open(dest_file, "wb") as out:
    out.write(downloaded.read())
    print("Done %s" % dest_file)

!pip3 install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
    
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Copy/download this file, Is this legal?
fid = drive.ListFile({'q':"title='keras_kaggle_digits.ipynb'"}).GetList()[0]['id']
f = drive.CreateFile({'id': fid})
f.GetContentFile('keras_kaggle_digits.ipynb')

[====================================================================================================] 100%processing test.csv data
Done /content/datalab/test.csv
[====================================================================================================] 100%processing train.csv data
Done /content/datalab/train.csv


In [0]:
#Define Model
def create_model(x_train, y_train, x_test, y_test):
    
    
    #input layer
    model = keras.Sequential()

    #convolution layer 1
    model.add(keras.layers.Conv2D({{choice([32, 40, 50, 64])}}, {{choice([(3,3),(5,5)])}}, input_shape=(28, 28, 1), activation={{choice([tf.nn.elu, tf.nn.relu, tf.nn.leaky_relu])}}))

    #pool layer 1
    #input [batch_size, 24, 24, 20]
    #output [batch_size, 12, 12, 20]
    model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

    #convolution layer 2
    model.add(keras.layers.Conv2D({{choice([32, 40, 50, 64])}}, {{choice([(3,3),(5,5)])}}, activation={{choice([tf.nn.elu, tf.nn.relu, tf.nn.leaky_relu])}}))
   
    
    #pool layer 2
    #input [batch_size, 8, 8, 40]
    #output [batch_size, 4, 4, 40]
    model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
    
    #pool layer 3
    if {{choice(['add','dont'])}} == 'add':

        #convolution layer 2
        model.add(keras.layers.Conv2D({{choice([32, 40, 50, 64])}}, (3,3), activation={{choice([tf.nn.elu, tf.nn.relu, tf.nn.leaky_relu])}}))
        model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(keras.layers.Flatten())
    
    #dense layer 1
    model.add(keras.layers.Dense({{choice([50, 100, 150, 200])}}, activation={{choice([tf.nn.elu, tf.nn.relu, tf.nn.leaky_relu])}}))
    
    #dropout layer
    model.add(keras.layers.Dropout({{uniform(0,1)}}))
    
    #dense layer 2
    model.add(keras.layers.Dense({{choice([50, 100, 150, 200])}}, activation={{choice([tf.nn.elu, tf.nn.relu, tf.nn.leaky_relu])}}))
    
    #dropout layer
    model.add(keras.layers.Dropout({{uniform(0,1)}}))
    
    #output layer
    model.add(keras.layers.Dense(10, activation=tf.nn.softmax))       
    
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'],
                  optimizer={{choice(['rmsprop', 'adam', 'adadelta', 'sgd', 'RMSprop'])}})

    
    train_datagen = keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        rotation_range=30,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        fill_mode='constant',
        cval=0,
        horizontal_flip=True)

    batch_size = 1
    
    if {{choice([64, 128])}} == 64:
        batch_size = 64
    else:
        batch_size = 128
    
    train_generator = train_datagen.flow(
        x_train, y_train,  
        batch_size=batch_size
    ) 


    
    model.fit_generator(train_generator,
              steps_per_epoch=42000 // batch_size,
              epochs=1,
              verbose=2)
    score, acc = model.evaluate(x_test / 255, y_test, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}




In [0]:
#Get input
def data():
    import numpy as np
    data = np.loadtxt("/content/datalab/train.csv", skiprows=1, delimiter=',')
    np.random.shuffle(data)

    features = np.reshape(data[:, 1:], (-1, 28, 28))

    features = features.astype('float32')

    labels = np.reshape(data[:, 0], (-1))
    labels = labels.astype(np.int32)
    expanded_labels = labels

    
    expanded_features = features.reshape(-1, 28, 28, 1)
    
    num_elements = expanded_features.shape[0]
    x_train = expanded_features[:int(num_elements * .95)]
    x_test = expanded_features[int(num_elements * .95):]
    y_train = expanded_labels[:int(num_elements * .95)]
    y_test = expanded_labels[int(num_elements* .95):]

    y_train = keras.utils.to_categorical(y_train, 10)
    y_test = keras.utils.to_categorical(y_test, 10)
    
    return x_train, y_train, x_test, y_test


In [5]:
from hyperopt import Trials, STATUS_OK, tpe


from hyperas import optim
from hyperas.distributions import choice, uniform

#Get input
test_data = np.loadtxt("/content/datalab/test.csv", skiprows=1, delimiter=',')
test_features = np.reshape(test_data, (-1, 28, 28, 1))
                          

Using TensorFlow backend.


In [6]:
best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=20,
                                          trials=Trials(),
                                          notebook_name='keras_kaggle_digits')


>>> Imports:
#coding=utf-8

try:
    import tensorflow as tf
except:
    pass

try:
    from tensorflow import keras
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    from google.colab import files
except:
    pass

try:
    from google.colab import auth
except:
    pass

try:
    from googleapiclient.discovery import build
except:
    pass

try:
    import io, requests, os
except:
    pass

try:
    import sys
except:
    pass

try:
    from googleapiclient.discovery import build
except:
    pass

try:
    from googleapiclient.http import MediaIoBaseDownload
except:
    pass

try:
    from pydrive.auth import GoogleAuth
except:
    pass

try:
    from pydrive.drive import GoogleDrive
except:
    pass

try:
    from google.colab import auth
except:
    pass

try:
    from oauth2client.client import GoogleCredentials
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from 

 - 18s - loss: 2.2114 - acc: 0.1755
Test accuracy: 0.6133333333333333
Epoch 1/1
 - 15s - loss: 2.3211 - acc: 0.1069
Test accuracy: 0.22428571428571428
Epoch 1/1
 - 17s - loss: 2.3040 - acc: 0.1091
Test accuracy: 0.10142857144276302
Epoch 1/1
 - 18s - loss: 2.1482 - acc: 0.1967
Test accuracy: 0.5871428569157918
Epoch 1/1
 - 16s - loss: 2.1954 - acc: 0.1698
Test accuracy: 0.4376190473919823
Epoch 1/1
 - 17s - loss: 1.8557 - acc: 0.3258
Test accuracy: 0.7933333333333333
Epoch 1/1
 - 16s - loss: 1.0360 - acc: 0.6446
Test accuracy: 0.9061904761904762
Epoch 1/1
 - 16s - loss: 0.9388 - acc: 0.6787
Test accuracy: 0.9328571428571428
Epoch 1/1
 - 19s - loss: 1.5014 - acc: 0.4637
Test accuracy: 0.8352380951245626
Epoch 1/1
 - 16s - loss: 1.3117 - acc: 0.5477
Test accuracy: 0.876666666553134
Epoch 1/1
 - 16s - loss: 1.2044 - acc: 0.5829
Test accuracy: 0.884285714058649
Epoch 1/1
 - 17s - loss: 1.3077 - acc: 0.5404
Test accuracy: 0.857619047505515
Epoch 1/1
 - 17s - loss: 1.0382 - acc: 0.6426
Test 

In [7]:
x_train, y_train, x_test, y_test = data()

print(best_run)

train_datagen = keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        rotation_range=30,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        fill_mode='constant',
        cval=0,
        horizontal_flip=True)
    

batch_size = [64, 128][best_run['optimizer_1']]
    
train_generator = train_datagen.flow(
        x_train, y_train,  
        batch_size=batch_size
    ) 

test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)


validation_generator = test_datagen.flow(
        x_test, y_test,
        batch_size=batch_size,
)


best_model.fit_generator(train_generator,
          steps_per_epoch = 42000 // batch_size,
          epochs=50,
          verbose=2,
          validation_data=validation_generator)
predictions = best_model.predict(test_features)

print(predictions)

{'Conv2D': 1, 'Conv2D_1': 1, 'Conv2D_2': 1, 'Conv2D_3': 1, 'Conv2D_4': 3, 'Dense': 3, 'Dense_1': 3, 'Dropout': 0.2855072647865474, 'Dropout_1': 0.5371229457196108, 'activation': 2, 'activation_1': 1, 'activation_2': 1, 'activation_3': 0, 'activation_4': 2, 'activation_5': 2, 'optimizer': 1, 'optimizer_1': 1}
Epoch 1/50
 - 14s - loss: 0.4124 - acc: 0.8654 - val_loss: 0.1265 - val_acc: 0.9633
Epoch 2/50
 - 14s - loss: 0.2985 - acc: 0.9054 - val_loss: 0.0893 - val_acc: 0.9729
Epoch 3/50
 - 14s - loss: 0.2448 - acc: 0.9240 - val_loss: 0.0902 - val_acc: 0.9719
Epoch 4/50
 - 14s - loss: 0.2126 - acc: 0.9346 - val_loss: 0.0821 - val_acc: 0.9752
Epoch 5/50
 - 14s - loss: 0.1962 - acc: 0.9403 - val_loss: 0.0810 - val_acc: 0.9738
Epoch 6/50
 - 14s - loss: 0.1817 - acc: 0.9444 - val_loss: 0.0887 - val_acc: 0.9710
Epoch 7/50
 - 14s - loss: 0.1766 - acc: 0.9459 - val_loss: 0.0617 - val_acc: 0.9810
Epoch 8/50
 - 14s - loss: 0.1594 - acc: 0.9512 - val_loss: 0.0745 - val_acc: 0.9800
Epoch 9/50
 - 14s 

 - 14s - loss: 0.1096 - acc: 0.9668 - val_loss: 0.0504 - val_acc: 0.9843
Epoch 32/50
 - 14s - loss: 0.1042 - acc: 0.9678 - val_loss: 0.0630 - val_acc: 0.9824
Epoch 33/50
 - 14s - loss: 0.1064 - acc: 0.9678 - val_loss: 0.0546 - val_acc: 0.9843
Epoch 34/50
 - 14s - loss: 0.1041 - acc: 0.9672 - val_loss: 0.0496 - val_acc: 0.9829
Epoch 35/50
 - 14s - loss: 0.0994 - acc: 0.9696 - val_loss: 0.0517 - val_acc: 0.9810
Epoch 36/50
 - 14s - loss: 0.0993 - acc: 0.9690 - val_loss: 0.0554 - val_acc: 0.9824
Epoch 37/50
 - 14s - loss: 0.0961 - acc: 0.9711 - val_loss: 0.0566 - val_acc: 0.9800
Epoch 38/50
 - 14s - loss: 0.1006 - acc: 0.9690 - val_loss: 0.0676 - val_acc: 0.9795
Epoch 39/50
 - 14s - loss: 0.0985 - acc: 0.9700 - val_loss: 0.0607 - val_acc: 0.9843
Epoch 40/50
 - 14s - loss: 0.1005 - acc: 0.9694 - val_loss: 0.0514 - val_acc: 0.9833
Epoch 41/50
 - 14s - loss: 0.0963 - acc: 0.9709 - val_loss: 0.0479 - val_acc: 0.9857
Epoch 42/50
 - 14s - loss: 0.0956 - acc: 0.9704 - val_loss: 0.0587 - val_acc:

In [0]:
i = 1
with open('submission.csv', 'w') as file:
    file.write('ImageId,Label\n')
    for pred in predictions:
        file.write("{},{}\n".format(i, np.argmax(pred)))
        i += 1

from google.colab import files
files.download('submission.csv')